# SimDatabase

generate the database of self maded firmwares 

In [1]:
from utils.key import *
from utils.db import * 
from utils.launcher import HeadlessLoggingPyhidraLauncher

In [2]:
# start the Launcher 
launcher = HeadlessLoggingPyhidraLauncher(verbose=True, log_path='./launch.log')
launcher.start()

INFO  Using log config file: jar:file:/home/dingisoul/dev/ghidra_11.0.1_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: ./launch.log (LoggingInitialization)  
INFO  Loading user preferences: /home/dingisoul/.ghidra/.ghidra_11.0.1_PUBLIC/preferences (Preferences)  
INFO  Searching for classes... (ClassSearcher)  
INFO  Class search complete (629 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  


In [3]:
# Reuse the project create or open in chapter 1 
# Necessary imports for ghidra project 
from ghidra.base.project import GhidraProject
from java.io import IOException
from pathlib import Path 

# Create Project Dir and name 
project_location = Path('./ghidra_project')
project_location.mkdir(exist_ok=True, parents=True)
project_name = "database_project"

# create or open project 
try:
    project = GhidraProject.openProject(project_location, project_name, True)
    print(f'Opened project: {project.project.name}')
except IOException:
    project = GhidraProject.createProject(project_location, project_name, False)
    print(f'Created project: {project.project.name}')

INFO  Opening project: /home/dingisoul/dev/FirmFlaw/ghidra_project/database_project (DefaultProject)  
Opened project: database_project


In [5]:
import os 
import subprocess
from functools import reduce
from operator import mul

# define the move function 
def save_append():
    return 

# configure the path
dir_path = Path('./match_base/')
DATABASE = dir_path / 'func.db'
if not os.path.exists(DATABASE):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    sql_create_table(cursor,FUNC_KEYS,FUNC_TABLE_NAME)
    sql_create_index(cursor,FUNC_TABLE_NAME,['hash'],'index_hash')
    conn.commit()
else: # append mode 
    print('APPEND MODE')
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    dir_path = Path('./match_base/append')
    if not os.path.exists(str(dir_path)):
        print("no append files")
        exit(0)
    def save_append():
        print(f'save append files in {dir_path}')
        subprocess.Popen('mv ./* ../', shell=True, cwd=dir_path)

# if no database create the table and index 
exist_files_ = []
for file_ in project.getRootFolder().getFiles():
    exist_files_.append(file_.getName())

num = 0
# add all function in file from match_base to database 
for root, dirs, files in os.walk(dir_path):
    for file_ in files:
        # skip database
        program_path = Path(root) / file_
        if program_path == DATABASE:
            continue 
        print(f"\033[31mIter file {program_path} at {num}\033[0m")
        num += 1
        if file_ in exist_files_:
            program = project.openProgram('/', file_, True)
        else:
            program = project.importProgram(program_path)
            # get the flat api 
            from ghidra.program.flatapi import FlatProgramAPI
            flat_api = FlatProgramAPI(program)
            flat_api.analyzeAll(program)
        rows_ = []
        for func_ in program.getListing().getFunctions(True):
            if filter_func(func_):
                row_ = (func_.getName(),program.getName())
                inst_ = get_inst_key(func_)
                graph = get_struct_graph_key(func_)
                # make sure the inst_[0] means the numAddress 
                hash_ = reduce(mul,(n for n in graph),1) * inst_[0]
                #if hash_ >= 0xffffffff:
                #   print(f'WARNING: {func_.getName()} hash is a little long {hash_}')
                row_ += (hash_,) + inst_ + graph
                if not sql_check_duplicate_func(cursor, row_[func_key_idx('name')], row_[func_key_idx('hash')], FUNC_TABLE_NAME):
                    rows_.append(row_)
        # insert the rows    
        sql_insert(cursor, FUNC_KEYS.keys(), rows_, FUNC_TABLE_NAME)
        conn.commit()
        print(f"\033[31mInsert {len(rows_)} Functions\033[0m")
        project.saveAs(program, "/", program.getName(), True)
        project.close(program)
# close the connect 
conn.close()
save_append()
project.close()

APPEND MODE
Iter file match_base/append/hardwario/twr-radio-door-sensor.elf at 0
INFO  Starting cache cleanup: /tmp/dingisoul-Ghidra/fscache2 (FileCacheMaintenanceDaemon)  
INFO  Finished cache cleanup, estimated storage used: 0 (FileCacheMaintenanceDaemon)  
INFO  Using Loader: Executable and Linking Format (ELF) (AutoImporter)  
INFO  Using Language/Compiler: ARM:LE:32:v8:default (AutoImporter)  
INFO  Packed database cache: /tmp/dingisoul-Ghidra/packed-db-cache (PackedDatabaseCache)  
INFO  Applied data type archive: generic_clib (ApplyDataArchiveAnalyzer)  
INFO  -----------------------------------------------------
    ARM Constant Reference Analyzer            1.204 secs
    ARM Symbol                                 0.002 secs
    ASCII Strings                              0.164 secs
    Apply Data Archives                        0.513 secs
    Call Convention ID                         0.107 secs
    Call-Fixup Installer                       0.007 secs
    Create Address Table

In [6]:
print('END')

END


In [ ]:
# TODO: add human defined database to classify the rtos, library and the crypto functions
# TOOD: add psuedo codes and decompiled c string comparison

In [13]:
subprocess.Popen('mv ./* ../', shell=True, cwd=dir_path)

<Popen: returncode: None args: 'mv ./* ../'>